In [ ]:
from read_data import get_training, get_test, get_sparse
from processing import exclude_non_numeric
from utils import create_csv_output
import numpy as np
import pandas as pd
import scipy
import pickle
from scipy.sparse import csr_matrix
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split, cross_val_score, ShuffleSplit, StratifiedShuffleSplit
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
np.random.seed(30027)
sns.set(rc={"figure.facecolor": "white"})

In [ ]:
from sklearn.ensemble import StackingClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import ComplementNB

## Train data

In [ ]:
train_X,train_y = get_training()
train_X_numeric = csr_matrix(FunctionTransformer(np.log1p).fit_transform(exclude_non_numeric(train_X)))
train_name, train_ingr, train_steps = get_sparse(data="train")
train_sparse = scipy.sparse.hstack((train_X_numeric, train_name, train_ingr, train_steps), format="csr")

In [ ]:
# Change target from (1.0, 2.0, 3.0) -> (0, 1, 2)
y = (train_y - 1).astype(int)
X_train, X_test, y_train, y_test = train_test_split(train_sparse, y, test_size=0.2)

## Test data

In [ ]:
test_X = get_test()
test_X_numeric = csr_matrix(FunctionTransformer(np.log1p).fit_transform(exclude_non_numeric(test_X)))
test_name, test_ingr, test_steps = get_sparse(data="test")
test_sparse = scipy.sparse.hstack((test_X_numeric, test_name, test_ingr, test_steps), format="csr")

## Stacking
Using LightGBM, LogisticRegression, SGDClassifier, ComplementNB

In [ ]:
models = [
    ("LR", LogisticRegression()),
    ("CNB", ComplementNB()),
    ("LGBM", LGBMClassifier()),
    ("SGD", SGDClassifier())
]

In [ ]:
%%time
partial_stack = StackingClassifier(estimators=models, final_estimator=LogisticRegression()).fit(X_train, y_train)
pickle.dump(partial_stack, open("report_models/partial/stacking.sav", "wb"))

In [ ]:
pickle.dump(partial_stack, open("report_models/partial/stacking.sav", "wb"))

In [ ]:
cross_val_score(clf, X_test, y_test, cv=StratifiedShuffleSplit(n_splits=5, test_size=0.2))
# array([0.810625, 0.80625 , 0.809375, 0.794375, 0.805625])

### Analysis

In [ ]:
stack_score = partial_stack.score(X_test, y_test)
stack_pred = partial_stack.predict(X_test)
stack_cm = confusion_matrix(y_test, stack_pred, normalize='true')

In [ ]:
plt.figure(figsize=(5, 5))
sns.heatmap(stack_cm, annot=True, square=True, xticklabels=[1.0, 2.0, 3.0], yticklabels=[1.0, 2.0, 3.0], cmap="Blues_r")
plt.ylabel('Actual label')
plt.xlabel('Predicted label' + '\n\nAccuracy Score: {0}'.format(stack_score))
plt.title('Stacking Confusion Matrix', size=13)
plt.savefig("report_pics/stack_cm.png")

### Submission model

In [ ]:
%%time
stacking = StackingClassifier(estimators=models, final_estimator=LogisticRegression()).fit(train_sparse, y)
pickle.dump(stacking, open("report_models/submission/stacking.sav", "wb"))
# CPU times: user 1min 46s, sys: 2.25 s, total: 1min 48s
# Wall time: 1min 51s

In [ ]:
create_csv_output("report_submission/stacking", (stacking.predict(test_sparse) + 1).astype(float))